# Tensors using tensorflow

More specifically , we're going to cover :
- Intro to tensors
- Getting info from tensor
- Manipulating tensors
- Tensors & NumPy
- Using @tf.function (a way to speed up your regular Python Function)
- Using GPUs with TensorFlow (or TPUs)
- Exercise to try for yourself

# Introduction to TensorFlow

In [2]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [3]:
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:

vector = tf.constant([10 , 20])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 20])>

In [5]:
print(scalar.ndim)
print(vector.ndim)

0
1


In [6]:
matrix = tf.constant([[10 , 20],
                     [30,40]])

In [7]:
print(matrix)
print(matrix.ndim)

tf.Tensor(
[[10 20]
 [30 40]], shape=(2, 2), dtype=int32)
2


In [8]:
threeDTensor = tf.constant(4 , shape = (4,3,2) , dtype = tf.float16 )
print(threeDTensor)
print(threeDTensor.ndim)

tf.Tensor(
[[[4. 4.]
  [4. 4.]
  [4. 4.]]

 [[4. 4.]
  [4. 4.]
  [4. 4.]]

 [[4. 4.]
  [4. 4.]
  [4. 4.]]

 [[4. 4.]
  [4. 4.]
  [4. 4.]]], shape=(4, 3, 2), dtype=float16)
3


What we have created so far :
* Scalar : a single number
* Vector : a number with direction
* Matrix : a 2-dimentional array of numbers
* Tensor : a n-dimentional array of numbers
* In fact all are tensors with different dimentions .

## tf.variable
The Variable() constructor requires an initial value for the variable, which can be a Tensor of any type and shape. This initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed

In [9]:
var = tf.Variable([10,4])
var

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  4])>

In [10]:
var.assign([12,5])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([12,  5])>

In [11]:
var.assign_add([2,3])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([14,  8])>

🗝️ Which one should you use? tf.constant() or tf.Variable()?

It will depend on what your problem requires. However, most of the time, TensorFlow will automatically choose for you (when loading data or modelling data).

## Creating random tensors

Random tensor are tensors of random numbers .

### tf.random.uniform
Outputs random values from a uniform distribution.

### tf.random.normal
Outputs random values from a normal distribution.

In [12]:
tf.random.set_seed(5) # to create a reproducible sequence of tensors across multiple calls.
rand1 = tf.random.uniform(shape = (3,4) , minval = 2 , maxval = 4 , dtype = tf.float32)
tf.random.set_seed(5)
rand2 = tf.random.uniform(shape = (3,4) , minval = 2 , maxval = 4 , dtype = tf.float32)

rand1 , rand2 , rand1 == rand2

(<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[3.2527862, 3.0596864, 3.5169144, 3.0169768],
        [2.6883075, 2.6391954, 3.3833   , 2.5330508],
        [2.074568 , 2.4039161, 2.5551438, 2.2902877]], dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[3.2527862, 3.0596864, 3.5169144, 3.0169768],
        [2.6883075, 2.6391954, 3.3833   , 2.5330508],
        [2.074568 , 2.4039161, 2.5551438, 2.2902877]], dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=bool, numpy=
 array([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]])>)

In [13]:
tf.random.set_seed(2)
rand3 = tf.random.normal(shape = (1,) , mean = 2 , stddev = 3 , dtype = tf.float32 )
rand3
tf.random.set_seed(2)
rand4 = tf.random.normal(shape = (3,4) , mean = 2 , stddev = 3 , dtype = tf.float32)
rand3 , rand4 , rand3 == rand4

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.3085065], dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[ 3.3085065 , -3.7281384 ,  6.1367197 , -1.1217556 ],
        [ 1.9971324 ,  2.508434  ,  4.746893  ,  3.6537461 ],
        [ 1.5950904 ,  1.8138397 ,  0.91279435,  2.0708082 ]],
       dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=bool, numpy=
 array([[ True, False, False, False],
        [False, False, False, False],
        [False, False, False, False]])>)

## Seeding
1. Global Seed
2. Operational-level Seed

In [40]:
# 1. If neither the global seed nor the operation seed is set, we get different results for every call to the random op and every re-run of the program:
a = tf.random.normal((2,3) , 2 , 1)
b = tf.random.normal((2,3) , 2 , 1)
a , b

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1.8196933, 1.0497137, 1.9603595],
        [1.2574594, 3.3231523, 1.381452 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[2.2265286, 2.8106554, 2.7466094],
        [2.7749703, 2.7244873, 3.4217405]], dtype=float32)>)

In [41]:
# 2. If the global seed is set but the operation seed is not set, we get different results for every call to the random op, but the same sequence for 
# every re-run of the program:
tf.random.set_seed(5)
b = tf.random.normal((2,3) , 2 , 1 )
a = tf.random.normal((2,3) , 2 , 1 )

a , b

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[2.2265286, 2.8106554, 2.7466094],
        [2.7749703, 2.7244873, 3.4217405]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1.8196933, 1.0497137, 1.9603595],
        [1.2574594, 3.3231523, 1.381452 ]], dtype=float32)>)

In [16]:
tf.random.set_seed(5)
b = tf.random.normal((2,3) , 2 , 1 )
a = tf.random.normal((2,3) , 2 , 1 )

a , b

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[2.2265286, 2.8106554, 2.7466094],
        [2.7749703, 2.7244873, 3.4217405]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1.8196933, 1.0497137, 1.9603595],
        [1.2574594, 3.3231523, 1.381452 ]], dtype=float32)>)

In [17]:
# 3. If the operation seed is set, we get different results for every call to the random op, but the same sequence for every re-run of the program:
a = print(tf.random.uniform([1], seed=1))  # generates 'A1'
b = print(tf.random.uniform([1], seed=1))  # generates 'A2'
a , b

tf.Tensor([0.3875184], shape=(1,), dtype=float32)
tf.Tensor([0.98592305], shape=(1,), dtype=float32)


(None, None)

In [18]:
a = print(tf.random.uniform([1], seed=1))  # generates 'A1'
b = print(tf.random.uniform([1], seed=1))  # generates 'A2'
a , b

tf.Tensor([0.9647], shape=(1,), dtype=float32)
tf.Tensor([0.8333188], shape=(1,), dtype=float32)


(None, None)

## Shuffle the order of the elements in the tensor

In [19]:
tf.random.set_seed(5)
tf.random.shuffle(rand1 , seed = 5)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[2.6883075, 2.6391954, 3.3833   , 2.5330508],
       [2.074568 , 2.4039161, 2.5551438, 2.2902877],
       [3.2527862, 3.0596864, 3.5169144, 3.0169768]], dtype=float32)>

### Other ways to create tensors

In [20]:
tf.ones(shape = (3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [21]:
tf.zeros(shape = (3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### turn numpy into tensors
tensors can be be run on a GPU for faster calculation

In [22]:
import numpy as np

In [23]:
numpy_array = np.arange(0,12,dtype = np.int32)
tensor_array = tf.constant(numpy_array , shape = (3,4))
numpy_array , tensor_array

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
 array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])>)

## Getting information from a tensor
When dealing with you should beware of the following attributes :
* shape `tenspr.shape`
* rank `tensor.ndim`
* axis or dimention `tensor[0]` or `tensor[:,1]` - a particular dimention of a tensor
* size `tf.size(tensor)`

In [24]:
@tf.function
def tensor_operator(tensor):
    print("Datatype:", tensor.dtype)
    print("Number of dimention:", tensor.ndim)
    print("Shape:", tensor.shape)
    print("Element along 2 axis:", tensor.shape[2])
    print("Element along last axis:", tensor.shape[-1])

In [25]:
D4_tensor = tf.zeros(shape = (3,2,4,5) , dtype = tf.int16)
D4_tensor

<tf.Tensor: shape=(3, 2, 4, 5), dtype=int16, numpy=
array([[[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]]], dtype=int16)>

In [26]:
tensor_operator(D4_tensor)

Datatype: <dtype: 'int16'>
Number of dimention: 4
Shape: (3, 2, 4, 5)
Element along 2 axis: 4
Element along last axis: 5


In [27]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]])
tf.size(t).numpy()

12

##  Indexing and expanding tensors

In [28]:
# first two element of each dimention of D4_tensor 
D4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=int16, numpy=
array([[[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]],


       [[[0, 0],
         [0, 0]],

        [[0, 0],
         [0, 0]]]], dtype=int16)>

In [29]:
# Get the first element from each dim from each index except for the final one
D4_tensor[:2,:1,:1,:]

<tf.Tensor: shape=(2, 1, 1, 5), dtype=int16, numpy=
array([[[[0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0]]]], dtype=int16)>

In [30]:
rank2tensor = tf.constant([
    [3,5],
    [2,7]
])
rank2tensor , rank2tensor.ndim

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[3, 5],
        [2, 7]])>,
 2)

In [31]:
# to get the last item of each of rank of rank2tensor
rank2tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([5, 7])>

In [32]:
# add in extra dimention to our rank2tensor 
rank3tensor = rank2tensor[... , tf.newaxis]
rank3tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[3],
        [5]],

       [[2],
        [7]]])>

In [33]:
tf.expand_dims(rank2tensor , axis = -1) #-1 is last dim

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[3],
        [5]],

       [[2],
        [7]]])>

##  Manipulating tensors with basic operations

**Basic Operation**
`+`,`-`,`*`,`/`

In [34]:
tensor = tf.constant([[1,2] , [3,4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[11, 12],
       [13, 14]])>

In [35]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]])>

In [36]:
tensor * 3

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 3,  6],
       [ 9, 12]])>

In [37]:
tensor / 5

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[0.2, 0.4],
       [0.6, 0.8]])>

In [38]:
# we can use the tf built in function tool
tf.math.multiply(tensor , 3)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 3,  6],
       [ 9, 12]])>

In [39]:
tf.math.add(tensor , 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[11, 12],
       [13, 14]])>